## Script to show the number of times the scratchpad was looked at in every trial

### Should be edited to compute the total timing of each look and if the sentence was correct or not

In [1]:
%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import datetime
import re
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd


In [42]:
def FrequencyScratchPadPerTrial(UserLooksAtKey):
    
    ScratchPadAtnPhrase = dict()
    # assuming the first sentence is directly started, unless 'nextPhrase' is choosen
    nPhrase = 1
    nScratchPad = 0
    nRow = -1
    firstScratchPad = 0
    flagStart = 0
    
    for row in UserLooksAtKey:
        nRow = nRow + 1
        
        if row[1] == 'ScratchPad':
            
            # for consecutive scratchpad looks
            if UserLooksAtKey[nRow-1][1] == 'ScratchPad':
                if float(row[2]) == 0:
                    progressScratchPad = float(UserLooksAtKey[nRow-1][2])
                else:
                    progressScratchPad = float(row[2])
                    
            # if first time of the look at scratchpad
            else:
                # if not the very first look (means the variable for progress exists)
                if firstScratchPad > 0: 
                    if progressScratchPad > float(row[2]):
                        nScratchPad = nScratchPad + 1
                        ScratchPadAtnPhrase.update({nPhrase: nScratchPad})
                        # if progress is not 0, then update the progress --> sometimes, it is 0 at the end of the look
                        if float(row[2]) != 0:
                            progressScratchPad = float(row[2])
                            
                # if the very first look (means the variable for progress does not exist)    
                else:
                    nScratchPad = nScratchPad + 1
                    ScratchPadAtnPhrase.update({nPhrase: nScratchPad})                        
                    progressScratchPad = float(row[2])
                    firstScratchPad = 1
                    
        # if next phrase being typed                      
        if row[1] == 'NextPhrase':
            if float(row[2]) == 1: # if it is completely selected
                nPhrase = nPhrase + 1
                nScratchPad = 0 # reset to 0 for the next phrase

    return ScratchPadAtnPhrase
        

In [43]:
metricComputed = 'read_text_events_frequency_per_trial_ratio'
dataFolderName = r'C:\DTU\Data\201805_HealthnRehab\TypingData'
j = 0
scratchPadToTypingRatioFrequencyList = list()
Names = list()

for root, dirs, subfolder in os.walk(dataFolderName):

    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'notCompleted' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'tb' in root or 'joha' in root:
            continue
            
        scratchPad = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8')
                    readerScratchPad = csv.reader(fScratchPad)
                    scratchPad = list(readerScratchPad)
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the scratchpad log file')
            
            if 'js' in root:
                if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                    try:
                        fScratchPad = open(root + '\\' + file)
                        readerScratchPad = csv.reader(fScratchPad)
                        scratchPad = list(readerScratchPad)
                        scratchPad.remove(scratchPad[0])
                        #print(scratchPad)
                    except:
                        if fScratchPad is not None:
                            fScratchPad.close()
                        else:
                            print('error in opening the scratchpad log file')
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
        if scratchPad is None or userKeys is None:
            continue
        else:
            
            print(root)
                
            # Number of times scratchpad was looked at
            ScratchPadAtnPhraseDict = FrequencyScratchPadPerTrial(userKeys)
            print(ScratchPadAtnPhraseDict)
            
            # save only the name of the subject and the date part of the file
            a = re.compile('(?<=TypingData\\\May[0-9]{2}\\\)(.*)(?=\\\OptiKey)')
                
            Names.append(a.findall(root)[0])
            scratchPadToTypingRatioFrequencyList.append(ScratchPadAtnPhraseDict)
                
            #print(nScratchPad, nTypedChar)
                
            #nCount = nCount + 1
           
                
#if scratchPadToTypingRatioFrequencyList:
#    AddDataToFile(metricComputed, Names, scratchPadToTypingRatioFrequencyList)
                        

C:\DTU\Data\201805_HealthnRehab\TypingData\May15\akt_MS\OptiKeyLogs\2018-5-15-14-3-18
{1: 9, 2: 3, 4: 3, 5: 8, 6: 1}
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\be_DT\OptiKeyLogs\2018-5-15-14-51-13-notInclude
{1: 2, 2: 6}
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\jl_DT\OptiKeyLogs\2018-5-15-10-21-20
{2: 2}
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\KEA_MS\OptiKeyLogs\2018-5-15-12-31-49
{1: 2, 2: 1, 4: 2, 5: 4}
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\lone_DT\OptiKeyLogs\2018-5-15-11-4-33
{1: 11}
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\MK_DT\OptiKeyLogs\2018-5-15-14-32-48
{1: 11, 2: 3, 3: 2}
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\MT_MS\OptiKeyLogs\2018-5-15-10-52-23-notInclude
{1: 8, 2: 6, 3: 3, 4: 9}
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\ok_MS\OptiKeyLogs\2018-5-15-12-23-16
{1: 6, 3: 2, 5: 2}
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\pt_DT\OptiKeyLogs\2018-5-15-12-43-13
{1: 6, 2: 1, 4: 1, 5: 5}
C:\DTU\Data\201805_HealthnReh

In [36]:
ScratchPadAtnPhraseDict

0